In [3]:
using DelimitedFiles
include("graphFunctions.jl");

In [42]:
ct=readdlm("cosetTable",Int)
E,dE,F,d1,d2,g,dg=geometry(ct);
dist,prev=shortestPaths(g,1);
dist_dual,prev_dual=shortestPaths(dg,1);
path=shortestNonContractableCycle(E,dE,g,dg,d1,d2, dist, prev)
path_dual=shortestNonContractableCycle(dE,E,dg,g,transpose(d2),transpose(d1),dist_dual,prev_dual);

open("adjMatX", "w") do io
    writedlm(io, adjMatX(ct), ' ')
end;

open("dualLoop", "w") do io
    writedlm(io, [E[e] for e in path_dual], ' ')
end;

In [33]:
ct=readdlm("cosetTable",Int)
E=edges(ct);
am=adjMatX(ct)
redEdges=coloredEdges(E,am,1)
F=faces(ct);
compactifyEdges!(F,redEdges)
E,dE,d1,d2,g,dg=geometryFromFace(F)
dist,prev=shortestPaths(g,1);
shortestNonContractableCycle(E,dE,g,dg,d1,d2, dist, prev)

9-element Vector{Int64}:
  8
  9
 32
 31
 58
 57
 17
  2
  1